### Code: Takes input from the user and generates the appropriate json output.

In [ ]:
# Install necessary libraries if not already installed
!pip install groq together openai
#never mind this  is the best
import json
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디옴 4잔"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```


    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디움 4잔 1샷 추가
      -아이스 카페라떼 라지 2잔 휘핑크림 추가"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아이스 라떼 미디옴 1잔"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.
     Using the base promt
     if **Current Conversation History And Actions history**:
     {formatted_history}
     when **Customer's New Input**: "{user_input}"
     Based on this information, generate the appropriate natural language response and JSON output

    """
    print(" **Current Conversation History**:", formatted_history)
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Save natural language order summary to `summary_history`
    if natural_language_response:
        summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
        summary_history.append(summary_line)

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Add the latest user input and the response to the conversation history with labeled summary
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    #conversation_history.append(f"**Cumulative Order Summary So Far - Input {input_counter}**: {summary_line}")
    input_counter += 1

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        # Provide cumulative order summary as a natural response without JSON output
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        # Clear conversation history and summary history for a reset
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

# End of the session
print("Thank you for using the coffee kiosk!")


### Code: Takes input from the user and generates the appropriate dataset and json output

In [ ]:
# Install necessary libraries if not already installed
!pip install groq together openai
import json
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized
dataset = []               # Stores each interaction as a dataset entry

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디옴 4잔"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```


    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디움 4잔 1샷 추가
      -아이스 카페라떼 라지 2잔 휘핑크림 추가"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아이스 라떼 미디옴 1잔"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.
     Using the base promt
     if **Current Conversation History And Actions history**:
     {formatted_history}
     when **Customer's New Input**: "{user_input}"
     Based on this information, generate the appropriate natural language response and JSON output

    """
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main interaction loop
print("Welcome to the virtual coffee kiosk! What would you like to order?")
input_counter = 1

while not order_confirmed:
    # Take user input
    user_input = input("Customer: ")

    # Check if customer confirms the order
    if "주문 완료할게" in user_input:
        order_confirmed = True
        print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
        conversation_history.clear()
        summary_history.clear()
        continue

    # Generate the prompt based on the conversation history and new user input
    prompt = generate_prompt(conversation_history, user_input)

    # Get the model's response
    response = get_response(prompt)

    # Parse the model's response
    natural_language_response, json_output = parse_response(response)

    # Save natural language order summary to `summary_history`
    if natural_language_response:
        summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
        summary_history.append(summary_line)

    # Print the final natural language response for the customer
    print("Kiosk:", natural_language_response)

    # Add the latest user input and the response to the conversation history with labeled summary
    conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
    input_counter += 1

    # Special handling for order summary requests
    if "내가 지금까지 뭘 주문했지" in user_input:
        print("Kiosk:", "지금까지의 주문내역입니다:\n" + "\n".join(summary_history))

    # Handle order cancellation requests
    if "주문 취소" in user_input:
        print("Kiosk: 주문이 취소되었습니다. 새 주문을 시작해 주세요.")
        conversation_history.clear()
        summary_history.clear()
        input_counter = 1

    # Append interaction to dataset
    dataset.append({
        "instruction": "Process the customer’s order based on the latest input in the current conversation history, considering previous items ordered in the session, and generate the appropriate natural language response and JSON output.",
        "input": " ".join(conversation_history),
        "output": f"**Natural Language Response**: \"{natural_language_response}\"\n\n**JSON Output**:\n```json\n{json.dumps(json_output, indent=2)}\n```"
    })

# End of the session - print the dataset
print("Thank you for using the coffee kiosk!")
print("\nGenerated Dataset:\n")
print(json.dumps(dataset, indent=2, ensure_ascii=False))


### Code Automatisation: The code takes input from the file (line by line) same as the getting input for the user and generates the appropriate dataset and json output

In [ ]:
# Install necessary libraries if not already installed
!pip install groq together openai
import json
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized
dataset = []               # Stores each interaction as a dataset entry

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디옴 4잔"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```


    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디움 4잔 1샷 추가
      -아이스 카페라떼 라지 2잔 휘핑크림 추가"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아이스 라떼 미디옴 1잔"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.
     Using the base promt
     if **Current Conversation History And Actions history**:
     {formatted_history}
     when **Customer's New Input**: "{user_input}"
     Based on this information, generate the appropriate natural language response and JSON output
    """
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Main loop to read from instructions file
with open("instructions.txt", "r", encoding="utf-8") as file:
    input_counter = 1

    for line in file:
        user_input = line.strip()  # Read each line as customer input

        # Skip empty lines
        if not user_input:
            continue

        # Check if customer confirms the order
        if "주문 완료할게" in user_input:
            order_confirmed = True
            print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
            conversation_history.clear()
            summary_history.clear()
            continue

        # Generate the prompt based on the conversation history and new user input
        prompt = generate_prompt(conversation_history, user_input)

        # Get the model's response
        response = get_response(prompt)

        # Parse the model's response
        natural_language_response, json_output = parse_response(response)

        # Save natural language order summary to `summary_history`
        if natural_language_response:
            summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
            summary_history.append(summary_line)

        # Print the final natural language response for the customer
        print("Kiosk:", natural_language_response)

        # Add the latest user input and the response to the conversation history with labeled summary
        conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
        input_counter += 1

        # Append interaction to dataset
        dataset.append({
            "instruction": "Process the customer’s order based on the latest input in the current conversation history, considering previous items ordered in the session, and generate the appropriate natural language response and JSON output.",
            "input": " ".join(conversation_history),
            "output": f"**Natural Language Response**: \"{natural_language_response}\"\n\n**JSON Output**:\n```json\n{json.dumps(json_output, indent=2)}\n```"
        })

# End of the session - print the dataset
print("Thank you for using the coffee kiosk!")
print("\nGenerated Dataset:\n")
print(json.dumps(dataset, indent=2, ensure_ascii=False))


### Code Automatisation: The code takes input from the file (line by line) same as the getting input for the user and generates the appropriate dataset and json output.  ***Improved Dataset***

In [ ]:
# Install necessary libraries if not already installed
!pip install groq together openai
import json
from together import Together
from openai import OpenAI

# Initialize Together and OpenAI clients with your API keys
together_client = Together(api_key="8a7afb786e80870e79e094bb105c9fd6f9ace6acff04c80fcf4f71d18e1aecf4")
openai_client = OpenAI(api_key="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", base_url="https://api.perplexity.ai")

# Initialize conversation history, summary history, and order confirmation flag
conversation_history = []  # Stores each customer input as a string entry in the format: "Customer's X Input: [input]"
summary_history = []       # Stores each cumulative summary of orders
order_confirmed = False    # Tracks if the order is finalized
dataset = []               # Stores each interaction as a dataset entry

# Function to generate prompt based on conversation history and new input
def generate_prompt(conversation_history, user_input):
    # Format the conversation history and cumulative summary as a single string
    formatted_history = "\n".join(conversation_history) if conversation_history else "none"
    formatted_summary = "\n".join(summary_history) if summary_history else "none"

    base_prompt = f"""
    You are operating a virtual coffee kiosk that receives speech-to-text (STT) inputs from customers placing coffee orders. Your role is to understand and process these inputs, respond naturally in Korean, and generate a structured JSON file with the correct details for backend processing.

    **Key Requirements**:
    - **Menu Items**: The kiosk offers the following drinks:
        - Hot and Iced Coffee: 아메리카노, 라떼, 카푸치노, 카페모카, 바닐라라떼, 에스프레소, 카라멜마끼아또
        - Tea: 허브티, 홍차
        - Specialty Drinks: 초콜릿라떼, 레몬에이드, 복숭아아이스티, 딸기스무디, 망고스무디, 키위주스, 토마토주스
    - **Default Values**:
        - Use default size "미디움" and temperature "핫" only if the customer does not specify these details.
    - **Do Not Make Assumptions**:
        - If the customer specifies temperature or size, do not override it with defaults. For instance, if they say "아이스 라떼 두잔 주세요", the output should indicate "아이스" without changing it to "핫".

    **Customer Input and Expected Output Format**:
    - Each response should have:
      1. **Natural Language Confirmation**: Respond in Korean, starting with an action confirmation such as "[Drink] [quantity] 주문되었습니다." and follow with a full summary of all items ordered so far, beginning with "지금까지 주문하신 내용은 다음과 같습니다:".(also if there are any instance in history it should be added after this sentence)
      2. **Structured JSON Output**: Each JSON output should only contain the items directly requested in the latest input, not a full history.

    **JSON Output Format**:
    - The JSON should be structured as follows:
      ```json
      {{
          "action": "[action_type]",
          "order_items": [
              {{
                  "drink": "[Drink Name]",
                  "size": "[Size]",
                  "temperature": "[Temperature]",
                  "quantity": [Quantity],
                  "add_ons": [List of add-ons if any],
                  "extra_shots": [Number of extra shots if any]
              }}
          ]
      }}
      ```
      - **Example JSON Output**:
        ```json
        {{
            "action": "create_order",
            "order_items": [
                {{
                    "drink": "아메리카노",
                    "size": "미디움",
                    "temperature": "핫",
                    "quantity": 1,
                    "add_ons": [],
                    "extra_shots": 0
                }}
            ]
        }}
        ```

    **Available Actions for JSON Output**:
    - **create_order**: For new drink orders.
    - **add_item**: For adding a new item to the current order.
    - **modify_order**: For changing an existing item (e.g., modifying size or temperature).
    - **cancel_order**: To remove an order item or reset the order.
    - **recommend_closest_item**: If a requested item is unavailable, recommend the closest item.
    - **show_order_summary**: Display a summary of all items ordered so far.
    - **complete_order**: Finalize the order after confirmation.

    **Specific Scenarios and Expected Outputs**:
    - **Creating a New Order**:
      - **Customer Input**: "아메리카노 4잔 주세요."
      - **Natural Language Response**: "아메리카노 4잔 주문되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디옴 4잔"
      - **JSON Output**:
        ```json
        {{
          "action": "create_order",
          "order_items": [
            {{
              "drink": "아메리카노",
              "size": "미디움",
              "temperature": "핫",
              "quantity": 4,
              "add_ons": [],
              "extra_shots": 0
            }}
          ]
        }}
        ```


    - **Requesting Order Summary**:
      - **Customer Input**: "내가 지금까지 뭘 주문했지?"
      - **Natural Language Response**: "지금까지 주문하신 내용은 다음과 같습니다:
      -핫 아메리카노 미디움 4잔 1샷 추가
      -아이스 카페라떼 라지 2잔 휘핑크림 추가"
      - **JSON Output**: None (as it is just a summary request without any new action).

    - **Modifying an Existing Order**:
      - **Customer Input**: "주문한거 아이스 라떼로 바꿔줘."
      - **Natural Language Response**: "주문이 아메리카노에서 아이스 라떼로 변경되었습니다. 지금까지 주문하신 내용은 다음과 같습니다:
      -아이스 라떼 미디옴 1잔"
      - **JSON Output**:
        ```json
        {{
          "action": "modify_order",
          "old_drink": "아메리카노",
          "new_drink": "라떼",
          "size": "미디움",
          "temperature": "아이스",
          "quantity": 1,
          "add_ons": [],
          "extra_shots": 0
        }}
        ```

    - **Short Names or Misspellings**:
      - Recognize common shorthand or misspellings. For example:
        - "아아" should be interpreted as "아이스 아메리카노".
        - "뜨아" should be interpreted as "핫 아메리카노".

    - **Unavailable Items**:
      - If the customer requests an item not on the menu, respond politely and recommend a similar item if available.
      - **Example**:
        - **Customer Input**: "초코라떼 주세요."
        - **Natural Language Response**: "죄송합니다, 초코라떼는 메뉴에 없습니다. 대신 초콜릿라떼를 추천드립니다."
        - **JSON Output**:
          ```json
          {{
            "action": "recommend_closest_item",
            "requested_item": "초코라떼",
            "recommended_item": "초콜릿라떼"
          }}
          ```

    - **Order Confirmation**:
      - **Customer Input**: "주문 완료할게요."
      - **Natural Language Response**: "주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다."
      - **JSON Output**: Clear/reset for the next session.

    **Response Rules**:
    - Treat each new input as part of the same order until "주문 완료할게" is received, which finalizes the order.
    - Always confirm the latest action first in the natural language response, followed by a full order summary.
    - Ensure each JSON output reflects only the latest request, not the entire order history.
     Using the base promt
     if **Current Conversation History And Actions history**:
     {formatted_history}
     when **Customer's New Input**: "{user_input}"
     Based on this information, generate the appropriate natural language response and JSON output
    """
    return base_prompt.strip()

# Function to get response from Together API
def get_response(prompt):
    response = together_client.chat.completions.create(
        model="Bllossom/llama-3.2-Korean-Bllossom-3B",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.1,
        top_p=1,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>"]
    )
    return response.choices[0].message.content

# Function to parse response into natural language and JSON output
def parse_response(response):
    try:
        natural_language_response, json_output = response.split("JSON Output:")
        json_data = json.loads(json_output.strip())
        return natural_language_response.strip(), json_data
    except ValueError as e:
        print("Error parsing response:", e)
        return response, None

# Define expanded keyword lists for different instruction types
cancel_keywords = ["취소", "취소해주세요", "취소해줘", "취소할게요", "취소해", "삭제", "삭제해줘", "삭제할게요", "지워줘", "없애줘"]
change_keywords = ["바꿔", "바꿔주세요", "바꿔줘라", "바꿔라", "변경", "변경해", "변경해줘", "수정", "수정해줘", "고쳐줘", "수정할게"]
add_keywords = ["추가", "추가해", "추가해주세요", "더", "넣어줘", "넣어주세요", "더 추가", "더 넣어줘", "더 시켜줘", "더 주문"]
size_keywords = ["사이즈", "라지", "미디움", "엑스라지", "작은", "큰", "크게", "작게", "중간"]
temperature_keywords = ["아이스", "핫", "차갑게", "따뜻하게", "차가운", "뜨거운", "시원하게"]
completion_keywords = ["완료", "완료할게요", "끝낼게요", "끝내줘", "마칠게요", "이대로 끝", "확정해줘", "주문 끝", "주문 확정"]
summary_keywords = ["지금까지", "주문 내용", "내역", "뭘 주문했지", "요약", "현재 주문", "주문 상태", "주문 확인", "주문 보기"]

# Main loop to read from instructions file
with open("instructions.txt", "r", encoding="utf-8") as file:
    input_counter = 1

    for line in file:
        user_input = line.strip()  # Read each line as customer input

        # Skip empty lines
        if not user_input:
            continue

        # Check if customer confirms the order
        if "주문 완료할게" in user_input:
            order_confirmed = True
            print("Kiosk: 주문이 완료되었습니다. 결제는 카드리더기를 사용해주세요. 감사합니다.")
            conversation_history.clear()
            summary_history.clear()
            continue

        # Generate the prompt based on the conversation history and new user input
        prompt = generate_prompt(conversation_history, user_input)

        # Get the model's response
        response = get_response(prompt)

        # Parse the model's response
        natural_language_response, json_output = parse_response(response)

        # Save natural language order summary to `summary_history`
        if natural_language_response:
            summary_line = natural_language_response.split("\n", 1)[1]  # Extracts summary part
            summary_history.append(summary_line)

        # Print the final natural language response for the customer
        print("Kiosk:", natural_language_response)

        # Add the latest user input and the response to the conversation history with labeled summary
        conversation_history.append(f"Customer's {input_counter} Input: {user_input}")
        input_counter += 1

        # Determine instruction type based on user input
        if any(word in user_input for word in cancel_keywords):
            instruction = "Process an order cancellation."
        elif any(word in user_input for word in change_keywords):
            if any(word in user_input for word in size_keywords):
                instruction = "Process a modification for drink size."
            elif any(word in user_input for word in temperature_keywords):
                instruction = "Process an order with a modification request for temperature."
            else:
                instruction = "Process an order with extra shots or other modifications."
        elif any(word in user_input for word in summary_keywords):
            instruction = "Process a request for a summary of the current order."
        elif any(word in user_input for word in completion_keywords):
            instruction = "Complete the customer's order."
        elif "초코라떼" in user_input:
            instruction = "Process an order with a request for an unavailable item and suggest an alternative."
        elif "아이스" in user_input or "차가운" in user_input:
            instruction = "Process an order with a request for iced drinks."
        elif any(word in user_input for word in add_keywords):
            instruction = "Process an order with extra items or additional options."
        else:
            instruction = "Process a simple order with one or multiple drinks."

        # Create and append the dataset entry
        dataset.append({
            "instruction": instruction,
            "input": " ".join([f"Customer's {j+1} Input: {line.strip()}" for j, line in enumerate(conversation_history)]),
            "output": f"**Natural Language Response**: \"{natural_language_response.strip()}\"\n\n**JSON Output**:\n```json\n{json.dumps(json_output, indent=4, ensure_ascii=False)}\n```"
        })


# End of the session - print the dataset
print("Thank you for using the coffee kiosk!")
print("\nGenerated Dataset:\n")
print(json.dumps(dataset, indent=2, ensure_ascii=False))
